## Final Project

In [8]:
pwd

/scratch/moa2020


In [5]:
ln -s /scratch/work/courses/AppliedGenomicsSec3/groups/group5

In [6]:
ls -l group5

lrwxrwxrwx. 1 moa2020 moa2020 55 Apr 17 12:26 group5 -> /scratch/work/courses/AppliedGenomicsSec3/groups/group5


In [10]:
cd group5

## Data Importation From NCBI GEO

Downloading the fastq files for the 12 samples to be analyzed. This is done using a script which contains an array of numbers and the fasterq dump command from SRA tools. The option "split" is used here because the reads are paired end reads which need to be split in two before trimming.

In [12]:
touch import.sh

In [39]:
cat import.sh

#!/bin/bash

#SBATCH --cpus-per-task=8
#SBATCH --time=5:00:00
#SBATCH --mem=16GB
#SBATCH --array=0-11
#SBATCH --job-name=import
#SBATCH --output=import_slurm_%j.out

##########

# array contains list of numbers corresponsing to the last figures in SRR numbers of samples
# split-3 opition to split spots into biological reads.

#########

module load sra-tools/2.10.9
module load fastqc/0.11.9

# Creating the array for the SRR numbers
file_arr=(76 77 78 79 80 81 82 83 84 85 86 87)
echo ${file_arr[1]}

# To get fastq files and change permissions
fasterq-dump SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]} --split-3
chmod 777 *fastq

# To perform fastqc analysis
fastqc SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_1.fastq
fastqc SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_2.fastq

In [40]:
sbatch import.sh

Submitted batch job 32457027


In [47]:
ls -lh *fastq

-rwxrwxrwx+ 1 moa2020 moa2020 2.3G Apr 17 13:21 SRR9079176_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 3.8G Apr 17 13:21 SRR9079176_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 1.7G Apr 17 13:20 SRR9079177_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 2.8G Apr 17 13:20 SRR9079177_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 2.0G Apr 17 13:20 SRR9079178_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 3.4G Apr 17 13:20 SRR9079178_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 1.8G Apr 17 13:20 SRR9079179_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 3.0G Apr 17 13:20 SRR9079179_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 1.6G Apr 17 13:20 SRR9079180_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 2.7G Apr 17 13:20 SRR9079180_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 2.0G Apr 17 13:20 SRR9079181_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 3.3G Apr 17 13:20 SRR9079181_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 1.9G Apr 17 13:20 SRR9079182_1.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 3.1G Apr 17 13:20 SRR9079182_2.fastq
-rwxrwxrwx+ 1 moa2020 moa2020 2.0G Apr 17 13:20 SRR9079183_1.f

## Creating a directory for all SLURM output and fastqc analysis

In [49]:
mkdir all_slurm_output

In [50]:
mkdir fastqc_analysis

In [51]:
mkdir import

In [52]:
mv *out import

In [53]:
mv import all_slurm_output

In [54]:
mv *html fastqc_analysis

In [55]:
mv *zip fastqc_analysis

Upon inspection of the fastqc analysis results, none of the reads have sequences of poor quality. 

## Trimming Low-Quality Bases using Trimmomatic

Although none of the reads reported sequences of poor-quality, we will trim out low-quality bases as an extra quality control step.

In [57]:
touch trim.sh

In [8]:
cat trim.sh

#!/bin/bash

#SBATCH --cpus-per-task=8
#SBATCH --time=5:00:00
#SBATCH --mem=16GB
#SBATCH --array=0-11
#SBATCH --job-name=trim
#SBATCH --output=trim_slurm_%j.out

##########

# SBATCH Code accepts #two PE (Paired End)# fastq file.
# Output will  be 4 other fastq file with forward/ reverse, paired/unpaired
# From fastqc analysis, the following sequence lengths were observed for read 1: 14 nucleotides and read 2: 51 nucleotides
# Given the length of read 1, the minimum length for trimming was set to 10 nucleotides.
#########

module purge
module load trimmomatic/0.39

# Creating the array for the SRR numbers
file_arr=(76 77 78 79 80 81 82 83 84 85 86 87)

# Call Trimmomatic module for trim
java -jar $TRIMMOMATIC_JAR PE -threads 8 -phred33 SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_1.fastq SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_2.fastq \
SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_forward_paired.fastq SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_forward_unpaired.fastq \
SRR90791${file_arr[$SL

In [4]:
sbatch trim.sh

Submitted batch job 32795050


## Creating a directory for all trim SLURM output and adding them to existing SLURM outputs

In [12]:
mkdir trim

In [13]:
mv *out trim

In [14]:
mv trim all_slurm_output

## Sequence Alignment Using Hisat-2
The Human genome was obtained from https://www.ncbi.nlm.nih.gov/genome/guide/human/. The file was unzipped and an index was created. The trimmed paired reads were aligned to the reference genome to create SAM files using the Hisat-2 aligner.

In [5]:
gunzip GRCh38_latest_genomic.fna.gz

In [6]:
touch align.sh

In [8]:
cat align.sh

#!/bin/bash

#SBATCH --cpus-per-task=8
#SBATCH --time=5:00:00
#SBATCH --mem=16GB
#SBATCH --array=0-11
#SBATCH --job-name=alignment
#SBATCH --output=alignment_slurm_%j.out

##########

# SBATCH Code accepts two paired end files: forward (1) and reverse (2)
# $1 = reference genome: GRCh38.fa
# $2 = index prefix: human

#########

module purge
module load hisat2/2.2.1

# To build the reference genome index
hisat2-build $1 $2_genome

# Creating the array for the SRR numbers
file_arr=(76 77 78 79 80 81 82 83 84 85 86 87)

# To run alignment on the trimmed file
hisat2 -x $2_genome -1 SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_forward_paired.fastq -2 SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_reverse_paired.fastq \
-S SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}.sam -p 8


In [9]:
sbatch align.sh GRCh38_latest_genomic.fna h_sapiens

Submitted batch job 32850243


## Checking the output of the alignment. 
The flagstat command in the Samtools module will be used to inspect the quality of the alignment. If the percentage of mapped reads is less than 65% (as specified in our proposal), the alignent will be considered a failure.

In [ ]:
ls -lh *sam

In [17]:
touch inspect_alignment.sh

In [4]:
cat inspect_alignment.sh

#!/bin/bash

#SBATCH --cpus-per-task=8
#SBATCH --time=5:00:00
#SBATCH --mem=16GB
#SBATCH --array=0-11
#SBATCH --job-name=inspection
#SBATCH --output=inspection_slurm_%j.out

##########

# SBATCH Code analyzes the quality of the alignment performed.

#########

module purge
module load samtools/intel/1.14

# Creating the array
file_arr=(*sam)

# To inspect the output of the alignment
samtools flagstat ${file_arr[$SLURM_ARRAY_TASK_ID]}


In [5]:
sbatch inspect_alignment.sh

Submitted batch job 32859433


Upon inspection of the SAM files obtained from the sequence alignment using hisat-2, several SAM files reported 0 mapping and the highest mapping percent reported was 53.72%. Hence, the sequence alignment was considered a failure. We decided to use a different splice-aware sequence aligner, STAR. 

## Sequence Alignment Using STAR
The Human (GRCh38) reference genome and genome annotation file were obtained from https://www.ncbi.nlm.nih.gov/genome/guide/human/. The file was unzipped and an index was created. The trimmed paired reads were aligned to the reference genome to create SAM files using the STAR aligner.

In [20]:
gunzip GRCh38_latest_genomic.gff.gz

In [12]:
touch star_alignment.sh

In [24]:
# To create the directory where the reference genome will be stored
mkdir human_genome_index

In [2]:
cat star_alignment.sh

#!/bin/bash

#SBATCH --cpus-per-task=16
#SBATCH --time=24:00:00
#SBATCH --mem=48GB
#SBATCH --array=0-11
#SBATCH --job-name=Star_align
#SBATCH --output=star_alignment_slurm_%j.out

##########

# SBATCH Code accepts two paired end files: forward (1) and reverse (2)

# BUILDING THE REFERENCE GENOME
# --runThread: Number of threads (processors) for mapping reads to a genome
# --runmode: run mode for STAR. genomeGenerate mode builds genome index
# $1 = path to the directory (genomeDir) where the reference genome index will be stored: human_genome_index
# $2 = reference genome file in FASTA format (genomeFastaFiles): GRCh38_latest_genomic.fna
# $3 = GTF file for gene annotation (sjdbGTFfile): GRCh38_latest_genomic.gff
# --sjdbGTFtagExonParentTranscript: for defining the parent-child relationship when using GFF instead of GTF file
# --sjdbOverhang: length of reads around the splice junctions. The ideal values should be max(read length)-1: 50

# MAPPING THE READS TO THE GENOME
# --readFilesIn:

In [3]:
sbatch star_alignment.sh human_genome_index GRCh38_latest_genomic.fna GRCh38_latest_genomic.gff

Submitted batch job 32862501


In [25]:
squeue -u moa2020

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          32868955        cs ood-jupy  moa2020  R    1:12:35      1 cs116
        32862501_0        cs Star_ali  moa2020  R    9:17:53      1 cs146
        32862501_1        cs Star_ali  moa2020  R    9:17:53      1 cs155
        32862501_2        cs Star_ali  moa2020  R    9:17:53      1 cs194
        32862501_3        cs Star_ali  moa2020  R    9:17:53      1 cs292
        32862501_4        cs Star_ali  moa2020  R    9:17:53      1 cs377
        32862501_5        cs Star_ali  moa2020  R    9:17:53      1 cs379
        32862501_6        cs Star_ali  moa2020  R    9:17:53      1 cs398
        32862501_7        cs Star_ali  moa2020  R    9:17:53      1 cs403
        32862501_8        cs Star_ali  moa2020  R    9:17:53      1 cs409
        32862501_9        cs Star_ali  moa2020  R    9:17:53      1 cs456
       32862501_11        cs Star_ali  moa2020  R    9:17:53      1 cs075


In [21]:
seff 32862501_5

Job ID: 32862508
Array Job ID: 32862501_5
Cluster: greene
User/Group: moa2020/moa2020
State: RUNNING
Nodes: 1
Cores per node: 16
CPU Utilized: 00:00:00
CPU Efficiency: 0.00% of 5-20:17:36 core-walltime
Job Wall-clock time: 08:46:06
Memory Utilized: 0.00 MB (estimated maximum)
Memory Efficiency: 0.00% of 48.00 GB (48.00 GB/node)


## Inspecting the trimmed fastq files.

In [6]:
touch inspect_trimming.sh

In [9]:
cat inspect_trimming.sh

#!/bin/bash

#SBATCH --cpus-per-task=8
#SBATCH --time=5:00:00
#SBATCH --mem=16GB
#SBATCH --array=0-11
#SBATCH --job-name=inspect_trimming
#SBATCH --output=inspect_trimming_fastq_slurm_%j.out

##########

# This script performs fastqc analysis on the trimmed paired fastq files.

#########

module load fastqc/0.11.9

# Creating the array for the SRR numbers
file_arr=(76 77 78 79 80 81 82 83 84 85 86 87)
echo ${file_arr[1]}

# To perform fastqc analysis
fastqc SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_forward_paired.fastq
fastqc SRR90791${file_arr[$SLURM_ARRAY_TASK_ID]}_reverse_paired.fastq


In [10]:
sbatch inspect_trimming.sh

Submitted batch job 32869789


In [14]:
squeue -u moa2020

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          32868955        cs ood-jupy  moa2020  R      30:42      1 cs116
        32862501_0        cs Star_ali  moa2020  R    8:36:00      1 cs146
        32862501_1        cs Star_ali  moa2020  R    8:36:00      1 cs155
        32862501_2        cs Star_ali  moa2020  R    8:36:00      1 cs194
        32862501_3        cs Star_ali  moa2020  R    8:36:00      1 cs292
        32862501_4        cs Star_ali  moa2020  R    8:36:00      1 cs377
        32862501_5        cs Star_ali  moa2020  R    8:36:00      1 cs379
        32862501_6        cs Star_ali  moa2020  R    8:36:00      1 cs398
        32862501_7        cs Star_ali  moa2020  R    8:36:00      1 cs403
        32862501_8        cs Star_ali  moa2020  R    8:36:00      1 cs409
        32862501_9        cs Star_ali  moa2020  R    8:36:00      1 cs456
       32862501_11        cs Star_ali  moa2020  R    8:36:00      1 cs075


In [13]:
mkdir trimmed_fastqc_analysis

Upon our inspection of the fastqc analysis results for our paired trimmed files, there were no sequences flagged for poor quality.

## Creating a directory for all align SLURM output and adding them to existing SLURM outputs

In [34]:
mkdir star_alignment

In [36]:
mv *out star_alignment

## Creating a directory for all Fastq files 

In [27]:
mkdir trimmed_fastq_files

In [28]:
mv *paired.fastq trimmed_fastq_files

In [29]:
mkdir raw_fastq_files

In [30]:
mv *fastq raw_fastq_files

In [31]:
mkdir all_fastq_files

In [32]:
mv trimmed_fastq_files all_fastq_files

In [33]:
mv raw_fastq_files all_fastq_files

## Creation of Read Count Matrix.
htseqcount is used to build a count matrix. The matrix contains information specifically about the gene counts as "exon" is specified as the feature to count. The gff3 features file was obtained from https://www.ncbi.nlm.nih.gov/genome/guide/human/. The GFF3 file will be unzipped and used to build the count matrix in addition to the sorted and indexed BAM file. We ensured to use a reference genome file and features file from the same source to prevent errors due to non-compatibility.

## Next Step: Importing the Count Matrix File.
We obtained the count matrix file from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE131391Merging. The matrix file is unzipped and will be read into R.

In [40]:
gunzip GSE131391_count_matrix.txt.gz

In [41]:
# Creating a directory for our R files
mkdir Final_Project_grp5